In [1]:
###This is a python implementation of the original MATLAB RMT code described in Cawse-Nicholson et al. (2013)

###The code accepts a spectroscopic image in the form rows x columns x bands
###And a noise covariance matrix 

In [2]:
import h5py
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from numpy import linalg as LA

import RMT


In [4]:
f = h5py.File('SHIFT_estnoise_output.mat') 
N = np.array(f['N2'])

f = h5py.File('SHIFT_site_coordN.mat')
siteN = np.array(f['N']);

f = h5py.File('SHIFT_site_coordE.mat')
siteE = np.array(f['E'])

In [5]:
import xarray as xr

dat = xr.open_dataset("reference://", engine="zarr", backend_kwargs={
    "consolidated": False,
    "storage_options": {"fo": "s3://dh-shift-curated/aviris/v1/gridded/zarr.json"}
})

In [6]:
#Filter out wavelengths with water absorptions
w = np.array(dat.wavelength)
goodbands = np.logical_or(np.logical_or(np.logical_and(w > 405, w < 1340), np.logical_and(w > 1460, w < 1800)), np.logical_and(w > 2050, w < 2450))

In [7]:
import warnings 
warnings. filterwarnings('ignore')

In [16]:
import itertools

winsize = 50
xrange = siteE[0]
yrange = siteN[0]
K1 = np.empty(len(xrange))

for rows in range(0,len(xrange)-1):
    dsub = dat.sel(x=slice(xrange[rows]-winsize,xrange[rows]+winsize), y=slice(yrange[rows]+winsize,yrange[rows]-winsize)).sel(time=dat.time[12], method="nearest")
    if np.any(dsub.x):
        refldat = dsub.reflectance.values[:,goodbands,:] 
        if np.max(refldat)>0:
            dims = refldat.shape
            img = np.empty([dims[1],dims[2],dims[0]])
            for re in range(0,dims[0]):
                img[:,:,re] = refldat[re,:,:] 
            temp = RMT.RMT(img,N)
            print(temp)
            K1[rows] = temp
        else:
            K1[rows] = 0
            print('0')
    else: 
        K1[rows] = 0
        print('0')

%store K1


8
7
8
7
9
6
7
8
7
9
7
8
9
0
0
0
0
0
9
8
8
8
0
0
7
7
8
9
9
10
0
0
8
9
8
0
7
7
10
0
0
0
7
8
6
8
7
7
7
7
7
9
9
8
8
7
7
7
7
7
8
8
8
7
6
10
8
8
8
5
6
6
7
0
8
7
7
0
0
7
7
8
0
0
0
0
0
0
0
0
0
0
0
0
7
7
10
0
7
7
7
9
0
0
0
0
8
7
8
9
8
8
0
0
0
0
0
8
7
9
8
9
6
8
10
9
10
7
7
6
7
9
7
9
9
9
9
9
8
9
7
6
9
6
7
7
7
7
9
8
6
9
8
9
7
6
9
7
6
8
9
9
9
10
9
9
8
10
9
8
8
8
9
7
8
9
9
7
7
10
8
9
9
9
8
8
7
8
6
8
9
8
8
8
8
8
8
7
6
6
8
7
7
6
7
7
7
7
7
7
8
8
7
7
8
6
7
7
6
7
6
6
6
9
12
7
7
5
6
6
8
0
0
0
7
7
8
0
0
7
8
0
9
9
9
9
8
8
9
9
9
9
9
9
9
7
8
7
6
9
9
10
7
6
7
9
8
7
9
5
6
6
8
8
8
7
8
8
8
8
8
8
9
7
9
7
9
7
8
7
8
8
8
8
6
8
9
7
6
8
8
7
7
8
7
7
6
8
8
7
7
7
7
8
7
9
7
7
8
7
7
6
7
7
8
7
8
8
10
7
7
6
8
8
6
8
7
9
9
8
7
7
6
8
9
6
7
7
7
8
7
5
7
7
9
8
7
7
7
9
9
8
0
0
0
0
0
0
0
0
7
7
7
6
7
6
7
8
6
5
6
6
6
7
6
6
7
7
6
7
7
6
8
8
9
6
5
7
8
8
8
9
9
8
7
10
9
8
9
6
6
7
7
7
7
7
8
8
8
8
10
8
8
12
7
8
8
8
7
6
6
6
8
9
11
9
8
9
Stored 'K1' (ndarray)


In [17]:
file = open('K_sites_50win_final.txt','w')
content = str(K1)
file.write(content)
file.close()

In [21]:
import itertools
import csv

winsize = 50
xrange = siteE[0]
yrange = siteN[0]
S1 = np.empty(len(xrange))

for rows in range(0,len(xrange)-1):
    dsub = dat.sel(x=slice(xrange[rows]-winsize,xrange[rows]+winsize), y=slice(yrange[rows]+winsize,yrange[rows]-winsize)).sel(time=dat.time[12], method="nearest")
    if np.any(dsub.x):
        refldat = dsub.reflectance.values[:,85,:] #band 85 is ~800nm
        S1[rows] = np.std(refldat)/np.mean(refldat)
        print(S1[rows])
    else: 
        S1[rows] = 0
        print('0')

%store S1


0.10760851204395294
0.04306153208017349
0.07736950367689133
0.15140897035598755
0.19160893559455872
0.15117955207824707
0.07598788291215897
0.15506471693515778
0.12538331747055054
0.09145721048116684
0.18309982120990753
0
0
0
0
0
0
0.15040776133537292
0.07370550185441971
0.12803830206394196
0.1482292115688324
0.13489596545696259
0.11073461174964905
0.142520472407341
0.10635490715503693
0.11728823930025101
0.17038430273532867
0.07651177048683167
0.1085117906332016
0.1062762662768364
0.09356406331062317
0.09213049709796906
0.17642083764076233
0.10953734815120697
0.08133214712142944
0.17868928611278534
0.08249349892139435
0.1329909861087799
0.1749132126569748
0.10981836169958115
0.16075147688388824
0.1459318995475769
0.11298365145921707
0.12486346811056137
0.11849100887775421
0.11950229853391647
0.10133583843708038
0.10896208137273788
0.19073137640953064
0.16474014520645142
0.1881369948387146
0.16142494976520538
0.18099422752857208
0.19264164566993713
0.16158698499202728
0.272936284542083

In [22]:
file = open('Std_sites_50win_final.txt','w')
content = str(S1)
file.write(content)
file.close()